# AXA Report - naBYLa Team

<i>Léo Tréguer - leotreguer@hotmail.com, 
Yann Nicolas - yannnico@gmail.com, Benoit Letournel - benoit.letournel@gmail.com</i>

## I / Data pre-processing and first look into the data

### 1 - Data importation and column selection

We decide to import only those 3 variables into our Jupyter Notebook : 
 - DATE, ASS_ASSIGNMENT as they are the only variables that we could extract from the submission
 - CSPL_RECEIVED_CALLS as it is exactly the value we are trying to predict

This selection also helped us have a slimmer document to work on than the original CSV file. (hopefully, as we would have been limited by our RAM limits) : 

### 2-  Data cleaning and sum of duplicates 

We checked that there were no NULL values in the labels. 
Then, we noticed that there were "duplicates" in the data : there were different values for the number of call for the same timeslot and assignment, as shown below. 

<img src="images/duplicates.jpg">

Thus, we did a pandas groupby and Sum for the same DATE and ASS_ASSIGNMENT to concatenate those duplicates.


### 3- Data analysis

Our first approach was to sum all the calls for all the assignments per day for the whole dataset, in order to have a first picture of the dataset, as shown below. This would also allow us to assess which weeks of data need to be predicted. These "missing" weeks are the ones where there is no data on the graph below. 

<img src="images/Total_per_day_2011_2012_2013.jpg",width=800,height=800>

We notice that there is a periodicity in the number of calls, as there seem to be more calls on a Monday than on a Sunday. Moreover, we see that there is a significant increase of the number of calls per day in the second half of 2013, compared to the rest of the dataset, from to the beginning of 2011 to the first half of 2013.

Then, we wanted to have a grasp of the number of calls for the 26 different assignments. Note : there are only 24 different assignments for which we have to predict the number of calls in the prediction file, so we will have to drop two useless assignements ('Gestion Amex','Evenements') from the dataset. Below is the sum of calls per month for every assignment.

<img src="images/Nb of monthly calls per category over 3 years.png",width=800,height=800>
<head>
  <title>HTML Reference</title>
</head>

We notice that there are many different behaviors for the number of calls for every assignment, with little apparent seasonality.
The three categories with the largest number of calls are the following : Téléphonie, Tech. Axa and Tech. Inter. The number of calls per day for the assignement "Téléphonie" more or less triple compared to the previous period, which explains the increase in the number of total calls highlighted above.
The graphs of the number of calls per day for the whole dataset for these three categories are shown below. 

<img src="images/Tech_Inter_2011_2012_2013.jpg",width=800,height=800>
<img src="images/Tech_AXA_2011_2012_2013.jpg",width=800,height=800>
<img src="images/Telephonie_2011_2012_2013.jpg",width=800,height=800>

As a conclusion, the training set and algorithm used to predict the numbers of calls can be totally different from one category to another : 
- Tech inter has a pic on August/September each year. We want to predict August 2013 we need to take into account August 2012
- Téléphonie has a sharp rise of number of calls after June 2013. Predicting from June depends less on the beginning of the year
- Tech. Axa is pretty stable in term of number of calls



## II / Feature engineering 

### 1 - Creation of Date features

We created new Date features by splitting the Timestamp provided in dataset, as shown below : 

    X['YEAR'] = X['DATE'].dt.year
    X['MONTH'] = X['DATE'].dt.month
    X['DAY_nb'] = X['DATE'].dt.day
    X['HOUR'] = X['DATE'].dt.hour
    X['MIN'] = X['DATE'].dt.minute
    X['TIME'] = X['DATE'].dt.time

This allowed us to train our model more easily than with the original timestamp format. We also added two new features to the time features :
-the day of the week : 0 for Monday, 1 for Tuesday,..
-The week of the year : 1 for the first week of the year, 2 for the second week of the year

    X['DAY'] = X['DATE'].dt.dayofweek
    X['WEEK'] = X['DATE'].dt.week

These new features would help our model “catch” the seasonality of call, through a single week (for example, there are usually more calls on a Monday than on a Sunday) and through a year (for example, there are usually fewer calls in summer, during the holidays).
    
#### Additional date features added : bank holidays: 

Due to the nature of the business, it seems a fare estimate to add a bank holiday feature in our training set. 
These days usually represent a decrease in the number of phone calls to the Axa call center. 


## III / Evaluation - CV pipeline 

The pipeline that we put in place is the following to cross validate our models is the following.  

| Fold      | Training period           |  CV week                    | Submission week
|-----------|---------------------------|-----------------------------|---------------------------
| 1         | 365 days before CV week   | 7 days from 21 dec 2012     | 7 days from 28 dec 2012
| 2         | 365 days before CV week   | 7 days from 26 jan 2013     | 7 days from 2 feb 2013
| ...       | ...                       | ...                         | ... 

Thus, for each week we are required to submit on the leaderboard, we would cv on a week before this submission week and train on one year before the cv week.
One week of cross-validation has been chosen due to the periodicity of the pattern on each week. 
Working with time series, we prefered to choose this pipeline over K-fold cv as we consider that there is a strong correlation between a measure on t and t+1.  


## IV / Learning 

### 1 - Ensemble methods

Having selected our features, we wish to start to train a first model on the dataset. 

As mentioned on the subject, the first direction that we took is to use CART, Random forests and Gradient Boosting Regressors. 
- from sklearn.tree import DecisionTreeRegressor
- from sklearn.ensemble import RandomForestRegressor
- from sklearn.ensemble import GradientBoostingRegressor

#### a_ Trees
A simple decision tree with depth of 30 gives us a null train error on all 12 training sets but gives us really high values on the CV sets, especially from month 6 (June 2013). 
At first, we are not surprised by this phenomenom because we are aware that CART does overfit pretty well. 

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>week 1</th>
      <th>week 2</th>
      <th>week 3</th>
      <th>week 4</th>
      <th>week 5</th>
      <th>week 6</th>
      <th>week 7</th>
      <th>week 8</th>
      <th>week 9</th>
      <th>week 10</th>
      <th>week 11</th>
      <th>week 12</th>
    </tr>
    <tr>

    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Train</th>
      <td>0e+00</td>
      <td>0e+00</td>
      <td>0e+00</td>
      <td>0e+00</td>
      <td>0e+00</td>
      <td>0e+00</td>
      <td>0e+00</td>
      <td>0e+00</td>
      <td>0e+00</td>
      <td>0e+00</td>
      <td>0e+00</td>
      <td>0e+00</td>
    </tr>
    <tr>
      <th>CV</th>
      <td>9e+02</td>
      <td>3e+02</td>
      <td>5e+07</td>
      <td>2e+06</td>
      <td>1e+06</td>
      <td>6e+41</td>
      <td>2e+44</td>
      <td>4e+42</td>
      <td>1e+43</td>
      <td>1e+43</td>
      <td>8e+48</td>
      <td>1e+11</td>
    </tr>
  </tbody>
</table>

#### b_ Random Forests 

On a second trial, we will examine Random Forests with 
- n_estimators = 10 (the number of trees in the forest)
- criterion : default Mean Squared Errors. Here we can't choose our Linex loss to measure the quality of a split

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>week 1</th>
      <th>week 2</th>
      <th>week 3</th>
      <th>week 4</th>
      <th>week 5</th>
      <th>week 6</th>
      <th>week 7</th>
      <th>week 8</th>
      <th>week 9</th>
      <th>week 10</th>
      <th>week 11</th>
      <th>week 12</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Train</th>
      <td>2e-02</td>
      <td>3e-02</td>
      <td>3e-02</td>
      <td>5e-02</td>
      <td>1e-01</td>
      <td>9e-01</td>
      <td>3e+03</td>
      <td>2e-01</td>
      <td>1e-01</td>
      <td>2e-01</td>
      <td>1e+00</td>
      <td>4e+01</td>
    </tr>
    <tr>
      <th>CV</th>
      <td>2e+01</td>
      <td>2e+03</td>
      <td>1e+00</td>
      <td>2e+10</td>
      <td>5e+04</td>
      <td>3e+31</td>
      <td>1e+01</td>
      <td>7e-01</td>
      <td>8e+01</td>
      <td>2e+02</td>
      <td>6e+12</td>
      <td>4e+02</td>
    </tr>
  </tbody>
</table>

The results are more promising as we do not overfit "as much" compared to the previous Tree models. RandomForest are usually very good when there is many features. At each nodes, the features are selected randomly to create the split. Knowing scikit learn has an even better handy tool for avoiding overfitting (GradientBoosting), we decide to select this latest.

#### c_ Gradient Boosting

To set up Gradient Boosting, we use ParameterGrid search isntead of a classic GridSearch. A we are working on the time series, each sample is not iid so we do not want to shuffle it with a k_fold cv. The best parameters found are : 
{'min_samples_split': 20, 'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 7, 'min_samples_leaf': 30 })

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>week 1</th>
      <th>week 2</th>
      <th>week 3</th>
      <th>week 4</th>
      <th>week 5</th>
      <th>week 6</th>
      <th>week 7</th>
      <th>week 8</th>
      <th>week 9</th>
      <th>week 10</th>
      <th>week 11</th>
      <th>week 12</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Train</th>
      <td>2e-01</td>
      <td>4e-01</td>
      <td>1e+00</td>
      <td>2e+00</td>
      <td>2e+01</td>
      <td>1e+01</td>
      <td>4e+07</td>
      <td>1e+08</td>
      <td>3e+01</td>
      <td>1e+01</td>
      <td>3e+01</td>
      <td>7e+01</td>
    </tr>
    <tr>
      <th>CV</th>
      <td>1e+01</td>
      <td>9e+02</td>
      <td>1e+00</td>
      <td>1e+06</td>
      <td>1e+03</td>
      <td>4e+31</td>
      <td>7e+03</td>
      <td>2e+00</td>
      <td>5e+01</td>
      <td>6e+01</td>
      <td>8e+08</td>
      <td>2e+01</td>
    </tr>
  </tbody>
</table>


As expected, we obtain better results with a Gradient Boosting and ParameterSearch. Nevertheless, the errors are still very high especially on June 2013 week. 

#### d_Errors investigation

We ploted errors for Random Forest for the two major categories in terms of errors :  (Téléphonie and Tech AXA)

We found out that : 

a) Errors tends to explode on a specific cross_validation week. This is due to the fact that our linex loss function can "explode" pretty fast if the trend of received calls change significantly. We can recall the graph displaying the number of call for Téléphonie over 3 years, we noticed that the number of calls skyrocked at mid-year in 2013. This can account for the major increase we can observe. 


<img src="images/Error_RF.png",width=400,height=400>

b) Secondly, we observe that errors are mainly centered on Mondays, meaning that the model does not make a clear distinction between a week-end and working days. 

<img src="images/Error_RF_DAY.png",width=400,height=400>


#### Conlusion : 
We had pretty shaky results due to the facts that the models is making mistakes in Mean Square Error but we then assess our errors in Linex. A tiny error for Gradient Boosting can become a significant one for Linex. We want then to come out with a solution that take our loss function directly into account.

### 2 - Optimizing the Linex loss function

On this section, we are going to try to reformulate the problem with taking into account the Linex loss function. 

#### a_ Optimization Formulation

Supposing that we  want to find a linear solution for our problem. We would have to minimize the function below : 

$ \min_{(\theta)} F(\theta) = \frac{1}{n} \sum_{i=1}^{n} linex(y_{i} , \theta^{\top}x_{i}) = \frac{1}{n}
\sum_{i=1}^{n} exp(\alpha (y_{i} - \theta^{\top}x_{i}) ) - \alpha(y_{i} - \theta^{\top}x_{i}) $

The derivative of this function is : 

$ \frac{\partial F}{\partial \theta} = \frac{\alpha}{n}
\sum_{i=1}^{n} [1 - exp(\alpha (y_{i} - \theta^{\top}x_{i})] x_{i}^{\top}  $

Having written those two functions, we will use a L-BFGS descent to find the minimum.

For a first trial, we are going to assume that our features are only the Hours&Minutes timestamps. 
For that we split our training data set per ASS_ASSIGNMENT & DAY. 
Eg. for a Monday on Téléphonie, the value retained for prediction will depend only on the Hour:Minute.

#### b_ Results 

With the same pipeline defined and a 365 days of training set before each submission week, we obtain an error of 1143. It was so far the best result we had as before it was "exploding" due to the nature of the loss function.

### 3 - XGBoost 

To overcome the mistakes made by the MSE on Trees/Random Forests and GradientBoosting we looked at XGBoost which allows the use of custom objectives. 

Thinking that XGBoost could simplify the problems, we faced several hurdles : 
- We were not able to install XGBoost on all our computers (only 1 over 3 had a Mac)
- We had many difficulties to parameter XGBoost so as to make it work properly
- Custom Loss worked only for Classification on our case


#### a_ Objectiv formulation for XGBoost

To do so we need to compute the gradient and the hessian (actually only the double derivative) of the loss.

The linex loss
$ F(x) = \frac{1}{n} \sum_{i=1}^{n} linex(y_{i} ,x_{i}) = \frac{1}{n}
\sum_{i=1}^{n} (exp(\alpha (y_{i} - x_{i}) ) - \alpha(y_{i} - x_{i}) - 1) $

with $ x_{i} $ being the predicted value 

The gradient of this function is define for each i by : 

$ \frac{\partial F}{\partial x_{i}} = \frac{\alpha}{n}
(1 - exp(\alpha (y_{i} - x_{i}))  $

The hessian of this function is define for each i by : 

$ \frac{\partial^{2} F}{\partial x_{i}^{2}} = \frac{\alpha^{2}}{n}
exp(\alpha (y_{i} - x_{i})  $

#### b_ XGBoost results

One of the issue of using this loss is the risk of overflow, expecially when the labels can differ from 1000. If we have the same initial value, we are bound to have an overflow. Therefore, we need an approximated first value not so far from the result (this is what hinted that we could find simple solutions).

Then, we had an implementation issue, using the default loss returned results similar to the gradient boosting above. But using our custom loss had the unfortunate effect to limit drastically the variance in the results, resulting in an overestimation for most of the data (and underestimation for the few high values), except on the month of June where we observed an underestimation for all the data.


### 4 - Final solution : Mixed of heuristic approach and Optimization of loss function 

As we had the best results with the l-bfgs approach so far, and the error only exploded for one result: "Téléphonie" and for the month of june, no model based on the past dates manages to predict such a growth in the number of calls.

A simple remark is that the growth happens suddenly and is a one time thing. And in our approach we compute our error on this specific week. Then given that the error is the linex we can see that taking the max on the last weeks (same day, same hour, same minute) can outperform most algorithms with the choice of feature we have so far (time feature).

#### a_ Feature choice

This is why for the "téléphonie" categorie we tried a different approach. We replaced the feature (time) by those two value :
- the max on the last weeks (same day, same hour, same minute)
- the min on the last weeks (same day, same hour, same minute)

As these features doesn't contain the days, hours ..., we separated the datasets in 4 : week days vs week-ends, and day vs night.

To test this approach we created only small datasets containing values from only a recent past, as the evolution of téléphonie seemed to cause issues, also we removed the week of June that as the one time growth, as it was seen as a cause of overestimation.

As we chose the linear regression method (as described in the part2), it seemed best not to add features not related to the values at this point.

#### b_ Results


This methods gives some results that looks pretty much always as an overestimation based mainly on the max and a little on the min.
As the datasets used were relatively small we achieved a convergence after only a few iterations:
Here are the evolution of the objective function for the 12 months on the datasets "week days and day" that had the maximum values and therefore the biggest potential for underestimation.

<img src="images/Objectives_Tele.png">

As we can see, we always converge towards a value around 10. But the convergence for June doesn't start really high as there are no values that are above 1000 in the datasets, and this is different for the next months. And even if in June we change the range of values, as we are based on the last weeks values, the errors on the submission set should still be relatively limited.





## Conclusion

As a conclusion, we were able to make errors drop significantly during all our progress without really reached the level of the top 10 on the leaderboard. Our difficulties to run XGBoost properly made things impossible for us to reach those levels. 

Nevertheless, with a more heuristic approach, separating categories and finding a minimizer of our prediction function, wa have succeeded to drop the objectives up to 47. 

In practice, we think the bump observed on Telephonie is unpredictable. We are not able to predict it unless someone familiar with the AXA phone center advises us about this significant change in pattern. The algorithm choosen should be as much as possible resilient to those kind of events. 



